In [ ]:
# Install Poetry
!curl -sSL https://install.python-poetry.org | python3 -

# Add Poetry to PATH for the current session
import sys
# sys.path.append("/root/.local/bin")

# !poetry env use python3

# Set up a new project directory
!mkdir my_poetry_project
%cd my_poetry_project

# Initialize a new Poetry project
!poetry init --no-interaction

!poetry add torch numpy transformers datasets

!poetry install

# Upgrade fsspec using poetry
# !poetry add --upgrade fsspec

!poetry run python -c "import torch; import transformers; print('Dependencies installed successfully!')"

Retrieving Poetry metadata

# Welcome to Poetry!

This will download and install the latest version of Poetry,
a dependency and package manager for Python.

It will add the `poetry` command to Poetry's bin directory, located at:

/root/.local/bin

You can uninstall at any time by executing this script with the --uninstall option,
and these changes will be reverted.

Installing Poetry (1.8.4)
Installing Poetry (1.8.4): Creating environment
Installing Poetry (1.8.4): Installing Poetry
Installing Poetry (1.8.4): Creating script
Installing Poetry (1.8.4): Done

Poetry (1.8.4) is installed now. Great!

To get started you need Poetry's bin directory (/root/.local/bin) in your `PATH`
environment variable.

Add `export PATH="/root/.local/bin:$PATH"` to your shell configuration file.

Alternatively, you can call Poetry explicitly with `/root/.local/bin/poetry`.

You can test that everything is set up by executing:

`poetry --version`

/content/my_poetry_project
/bin/bash: line 1: poetry: comman

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install torch numpy transformers datasets
# !pip install evaluate
%load_ext autoreload
%autoreload 2
# You will have already needed to install a few things; (python 3.9+) > pip install poetry ; poetry install torch numpy transformers datasets
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertForSequenceClassification, BertTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, pipeline, EarlyStoppingCallback
from datasets import load_dataset, Dataset
import evaluate  # Correct import for metrics
from scipy.special import softmax
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import label_binarize
import copy
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

accuracy = evaluate.load("accuracy")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# PRETRAINED_MODEL_PATH= AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")
PRETRAINED_MODEL_PATH = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)  # 11 is the number of topics you have

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Single Multi-class for Topic Modeling

In [ ]:
# Load the dataset
dataset = pd.read_csv('/content/drive/MyDrive/dailyinks/data/merged_df.csv')  # Load entire dataset

# Define the mapping of topic columns to numeric labels directly
topic_mapping = {
    'exercise': 0,
    'family': 1,
    'food': 2,
    'friends': 3,
    'god': 4,
    'health': 5,
    'love': 6,
    'recreation': 7,
    'school': 8,
    'sleep': 9,
    'work': 10
}

# Map topics in the `topic` column to their corresponding integer labels
dataset['labels'] = dataset['topic'].map(topic_mapping)

# Split the dataset into training+validation and testing sets
train_val_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Load pre-trained BERT model
topic_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=11,  # Number of topic labels
    id2label={i: label for i, label in enumerate(topic_mapping.keys())},  # Map integers to label names
    label2id=topic_mapping,  # Map label names to integers
    local_files_only=True  # Assuming you have the model locally
)

topic_model.to(device)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.model_max_length = 512

# Preprocessing function
def preprocess(row):
    tokenized = tokenizer(row["text"], truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = row["labels"]
    return tokenized

# Freeze BERT embeddings and certain encoder layers, leaving task-specific head trainable
for name, param in topic_model.bert.named_parameters():
    param.requires_grad = False

# RECOMMENDED: for more capacity to learn, make BERT pooler and at least one encoder layer trainable
for name, param in topic_model.bert.pooler.named_parameters():
    param.requires_grad = True

for name, param in topic_model.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True


# Compute metrics function
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probabilities = softmax(predictions, axis=1)  # Apply softmax to get class probabilities
    predicted_classes = np.argmax(probabilities, axis=1)  # Get predicted class indices

    # One-hot encode labels for ROC-AUC
    labels_one_hot = label_binarize(labels, classes=list(topic_mapping.values()))

    # Compute metrics
    accuracy = accuracy_metric.compute(predictions=predicted_classes, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predicted_classes, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=predicted_classes, references=labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=predicted_classes, references=labels, average="weighted")["f1"]
    roc_auc = roc_auc_score(labels_one_hot, probabilities, average="weighted", multi_class="ovr")  # One-vs-Rest

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "roc_auc": roc_auc}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)  # Stop if validation loss doesn't improve for 3 epochs

# Define training arguments
training_args = TrainingArguments(
    output_dir="checkpoints",
    overwrite_output_dir=True,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    logging_steps=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,  # This is required for EarlyStoppingCallback
    metric_for_best_model="eval_loss", # Specify the metric to use for early stopping
)

# Implement k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

dataset_df = train_val_data.copy()
all_metrics = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset_df)):
    print(f"Processing Fold {fold + 1}/{k}...")

    train_data = dataset_df.iloc[train_idx]
    val_data = dataset_df.iloc[val_idx]

    train_dataset = Dataset.from_pandas(train_data).map(preprocess)
    val_dataset = Dataset.from_pandas(val_data).map(preprocess)

    fold_training_args = copy.deepcopy(training_args)
    fold_training_args.output_dir = f"checkpoints/fold_{fold + 1}"

    fold_trainer = Trainer(
        model=topic_model,
        args=fold_training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
        callbacks=[early_stopping]
    )

    fold_trainer.train()
    fold_metrics = fold_trainer.evaluate()
    print(f"Metrics for Fold {fold + 1}: {fold_metrics}")
    all_metrics.append(fold_metrics)

# Average metrics across folds
average_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Average Metrics Across Folds:")
print(average_metrics)

# Standard deviation metrics across folds
std_metrics = {key: np.std([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
print("Standard Deviation Metrics Across Folds:")
print(std_metrics)

# Evaluate the model on the testing set
test_dataset = Dataset.from_pandas(test_data).map(preprocess)

final_metrics = fold_trainer.evaluate(test_dataset)
print("Final Metrics on Testing Set:")
print(final_metrics)

Processing Fold 1/5...


Map:   0%|          | 0/4102 [00:00<?, ? examples/s]

Map:   0%|          | 0/1026 [00:00<?, ? examples/s]

<ipython-input-104-0c6f54bb52c4>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  fold_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,1.603800,1.312402,0.644250,0.736033,0.644250,0.598334,0.934095
2,0.634200,0.566070,0.847953,0.847247,0.847953,0.842837,0.977840
3,0.512700,0.440225,0.870370,0.870482,0.870370,0.868940,0.983559
4,0.415200,0.414725,0.884990,0.884357,0.884990,0.884013,0.985207
5,0.543900,0.410580,0.885965,0.885451,0.885965,0.884957,0.985511


...

<ipython-input-104-0c6f54bb52c4>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  fold_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.073400,0.039366,0.991220,0.991343,0.991220,0.991202,0.999771
2,0.057600,0.043399,0.986341,0.986509,0.986341,0.986243,0.999803
3,0.061100,0.039579,0.987317,0.987488,0.987317,0.987215,0.999832
4,0.050200,0.037144,0.987317,0.987499,0.987317,0.987255,0.999844
5,0.031700,0.037176,0.987317,0.987499,0.987317,0.987255,0.999846


Metrics for Fold 5: {'eval_loss': 0.03714396432042122, 'eval_accuracy': 0.9873170731707317, 'eval_precision': 0.9874988720297185, 'eval_recall': 0.9873170731707317, 'eval_f1': 0.9872547076177535, 'eval_roc_auc': 0.9998437394347897, 'eval_runtime': 29.6284, 'eval_samples_per_second': 34.595, 'eval_steps_per_second': 0.304, 'epoch': 5.0}
Average Metrics Across Folds:
{'eval_loss': 0.18378937616944313, 'eval_accuracy': 0.9473615746683782, 'eval_precision': 0.9473315889831838, 'eval_recall': 0.9473615746683782, 'eval_f1': 0.9469631316486569, 'eval_roc_auc': 0.995045387000107, 'eval_runtime': 29.40064, 'eval_samples_per_second': 34.8846, 'eval_steps_per_second': 0.306, 'epoch': 5.0}


Map:   0%|          | 0/1282 [00:00<?, ? examples/s]

Final Metrics on Testing Set:
{'eval_loss': 0.4132130444049835, 'eval_accuracy': 0.9024960998439937, 'eval_precision': 0.9024369492741785, 'eval_recall': 0.9024960998439937, 'eval_f1': 0.9020777762406281, 'eval_roc_auc': 0.9891053356451315, 'eval_runtime': 36.4509, 'eval_samples_per_second': 35.171, 'eval_steps_per_second': 0.302, 'epoch': 5.0}


In [ ]:
# Define the path to save the model in Google Drive
save_path = '/content/drive/MyDrive/dailyinks/model/topic_model'

# Save the model and tokenizer
topic_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

import shutil

# Create a zip file of the saved model directory
shutil.make_archive('/content/topic_model', 'zip', save_path)

# Download the zip file
from google.colab import files
files.download('/content/topic_model.zip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
classifier = pipeline("text-classification",model=topic_model, return_all_scores=False, tokenizer=tokenizer, device=device)
classifier("I had a great day with my family and felt really happy.")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'family', 'score': 0.999677300453186}]